[Hinode X-Ray Telescope]:https://xrt.cfa.harvard.edu/

# Hinode-XRT: A Practical Guide to Data Extraction and Visualization

This notebook provides a step-by-step guide on how to download, process, and visualize solar observation data from the [Hinode X-Ray Telescope], making it accessible for research and analysis.

*Created by Joy Velasquez. Version 1 - February 29, 2024*

## Introduction

This notebook is designed as a preliminary guide to handle solar observation data specifically from the Hinode X-Ray Telescope (XRT). While this example does not directly utilize `xrtpy` functionalities, it lays the groundwork for handling and understanding XRT data, which can be further processed and analyzed using `xrtpy`.

Here's what we'll cover:
- **Downloading XRT Data**: Step-by-step instructions to download XRT data for a specified period of interest.
- **Data Filtering**: Use custom functions for filtering and refining the downloaded data.
- **Visualization and Movie Creation**: Explore methods to visualize the data and compile it into movie format, providing a dynamic view of solar observations.

Whether for research analysis or as a stepping stone to using `xrtpy` for more advanced applications, this guide aims to equip you with the fundamental skills necessary for effective data manipulation and visualization in solar physics. 


## Table of Contents
1. [Importing Necessary Packages](#importing-necessary-packages)
2. [Using Fido to Search and Download XRT Data](#using-fido-to-search-and-download-xrt-data)
3. [Inspecting the Downloaded Data](#inspecting-the-downloaded-data)
4. [Exploring Functions to Enhance Data Quality](#exploring-functions-to-enhance-data-quality)
   - 4.1 [Filtering FITS Files Through Selected Filter](#filtering-fits-files-through-selected-filter)
   - 4.2 [Filtering FITS Files Through Pixel Size](#filtering-fits-files-through-pixel-size)
   - 4.3 [Filtering FITS Files by Exposure Time](#filtering-fits-files-by-exposure-time)
5. [Navigating FITS Images](#navigating-fits-images)
6. [Removing Unwanted FITS Files](#removing-unwanted-fits-files)
7. [Visualizing Solar Dynamics: Creating a Movie from XRT FITS Data](#visualizing-solar-dynamics-creating-a-movie-from-xrt-fits-data)


### Importing Necessary Packages

Before we begin, let's import all the necessary Python packages and modules required for this notebook. Key packages include:

- `sunpy`: An open-source Python library for Solar Physics data analysis.
- `matplotlib`: A library for creating static, animated, and interactive visualizations in Python.
- `astropy`: Used for astronomy and astrophysics data processing.

These packages, among others, will enable us to download, inspect, filter, and visualize the XRT data effectively.


In [ ]:
# Standard library imports for file and temporary directory management
import os
import tempfile

# Counting occurrences of unique elements
# creating and managing images and videos
import imageio
import ipywidgets as widgets
import matplotlib.pyplot as plt

# SunPy library for solar physics data analysis
import sunpy.map

# Astropy units module for handling physical quantities
from astropy import units as u

# IPython and ipywidgets for interactive widgets in the notebook
from IPython.display import (
    Video,  # For embedding videos in the notebook
    clear_output,
    display,
)

# SunPy modules for querying and downloading solar data
from sunpy.net import Fido
from sunpy.net import attrs as a

# #******************************************
# #Command to ensure necessary libraries are installed (uncomment and run if needed)
# !pip install sunpy matplotlib imageio ipywidgets
# #******************************************


## Using Fido to Search and Download XRT Data

[SunPy]:https://sunpy.org/
[Fido]:https://docs.sunpy.org/en/stable/generated/api/sunpy.net.Fido.html#sunpy.net.Fido

### Searching for Hinode XRT Data

[Fido] is a unified interface provided by [SunPy] for querying and downloading data across various solar physics data sources and missions. In this section, we'll focus on using Fido to find and retrieve data from the Hinode X-Ray Telescope (XRT).

We'll walk through the process of defining a search query, which includes specifying a time range and selecting the instrument, to fetch targeted datasets effectively. 

In this section, we'll focus on the observation of [Active Region (AR) 13234](https://www.solarmonitor.org/region_pop.php?date=20230223&type=hxrt_flter&region=13234) by Hinode-XRT. Recorded on May 21, 2023, around 18:51 UT, this region, located near the solar limb, showcased active flaring activities, presenting an intriguing case for study.

For those interested in further exploring flare activities observed by the Hinode mission, the [XRT Flare Catalog](https://xrt.cfa.harvard.edu/flare_catalog/) offers a comprehensive database.


In [ ]:
# Define the time range of interest for solar observations
time_range = a.Time("2021-05-21 18:51:00", "2021-05-22 01:59:54")

# Specify the instrument as 'xrt' to search for Hinode X-Ray Telescope data
instrument = a.Instrument("xrt")


# This will return a catalog of available XRT data during the specified period
xrt_downloaded_files = Fido.search(time_range, instrument)

# Display the search results
print(xrt_downloaded_files)

#### Downloading Hinode XRT Data Using `Fido.fetch`

Having identified the desired Hinode XRT data with `Fido.search`, we proceed to download the datasets using the `Fido.fetch` function. This function accepts the search results as its input and facilitates the data download to a designated directory on your local system.

In the next code cell, we illustrate the utilization of `Fido.fetch` to procure our previously identified data. This demonstration includes specifying a target directory for the downloads. Additionally, we'll introduce handling techniques for managing the download outputs.

**Note**: If no directory is specified, `Fido.fetch` defaults to downloading files to a SunPy-managed directory. You can customize the download location by providing a path to the `path` parameter of `Fido.fetch`.


In [ ]:
# Note: Depending on the amount of data and network speed, this process can take some time.
# The `progress` parameter in `Fido.fetch` controls the display of the download progress bar.
# Setting `progress=False` disables the progress bar, which can be useful for cleaner output,
# especially when running this in a script or automated pipeline. By default, we keep it True
# for this interactive session to monitor the download progress.

# This will download the files to the default SunPy data directory or a specified path.
# Replace `xrt_downloaded_files` with your search results variable if different.
xrt_files_results = Fido.fetch(xrt_downloaded_files, progress=True)

# If you wish to specify a different download directory, you can do so by adding the `path` parameter:
# xrt_files_results = Fido.fetch(xrt_downloaded_files, path='/desired/download/directory/', progress=True)

XRT follows standard Flexible Image Transport System ([FITS](https://fits.gsfc.nasa.gov/)) file naming conventions.



## Inspecting the Downloaded Data

Once the download is complete, it's important to inspect the data to ensure that what we have collected is suitable for our analysis. This step involves checking the basic information about the downloaded files, such as their names, file sizes, and data types. This information can help us verify that the download was successful and that we have the correct data for our project.

We will demonstrate practical methods to retrieve and display these details using Python's standard utilities along with SunPy-specific functions.

In [ ]:
# total number of files downloaded using Fido
num_files = len(xrt_files_results)
print(f"Number of files downloaded: {num_files}")

# few examples of file names to understand the naming convention
print("\nSample file names:")
for file in xrt_files_results[:5]:
    print(file)

# Example of inspecting a single file from the downloaded dataset
# Here, we choose a file arbitrarily (the 85th file) for inspection
sample_file_index = 84  # Adjust the index as desired
sample_data = sunpy.map.Map(xrt_files_results[sample_file_index])

# Print information about the sample file
# This includes metadata like observation time, instrument, and data dimensions
print(f"\nSample data info for file number {sample_file_index + 1}:")
print(sample_data)

# Visualize the data from the sample file
# This step is crucial for a quick quality check and to familiarize with the data
plt.figure(figsize=(10, 8))
sample_data.peek()  # 'peek' method generates a quick-look plot of the data
plt.show()

Having successfully collected the Hinode-XRT data, we now have several exciting options at our disposal. We can analyze the FITS files to understand the solar phenomena, visualize the observations in image form ([Navigating FITS Images](#Navigating_FITS_Images)), or even create a dynamic movie ([Creating a Movie from XRT FITS Data](#Creating-a-Movie-from-XRT-FITS-Data)) showcasing a solar event. However, we can take further step towards cleaning up some of the data we have downloaded following the *Exploring Functions to Enhance Data Quality* sections below, which is **optional**. 


## Exploring Functions to Enhance Data Quality

In the following sections, I introduce a series of custom functions aimed at filtering and refining the downloaded Hinode-XRT data. These functions are designed to be user-friendly and accessible, serving as a practical toolset for both novice and experienced users. 



### Why Enhance Data Quality?

The Sun, as observed by Hinode-XRT, presents dynamic and complex phenomena. Filtering the data helps isolate specific events or features, ensuring that subsequent analyses or visualizations focus on the most relevant and high-quality data. These enhancements are **optional** but highly recommended to achieve a clearer, more meaningful understanding of the solar data collected.



### Filtering FITS Files Through Selected Filter

Identifying the type of data contained within Hinode XRT FITS files can be challenging, as the file titles do not provide sufficient information. To address this, I have developed a function that leverages `sunpy.map` to read the header information of each FITS file. This process allows us to identify and count the unique filters present within the dataset. 

To utilize this function, simply run the provided code in the next cell. When prompted, enter your desired filter criteria. This intuitive approach enables you to effectively narrow down your dataset to include only the most relevant observations for your analysis.

In [ ]:
def normalize_string(s):
    """
    Normalize a string for comparison by converting to lowercase, replacing hyphens and underscores with spaces, and removing 'open'.

    Parameters:
    - s (str): The string to normalize.

    Returns:
    - str: The normalized string.
    """
    return (
        s.lower()
        .replace("open", "")
        .strip("- ")
        .replace("_", " ")
        .replace("-", " ")
        .strip()
    )


## I have plans to update this function at a later time to allow users to collect FITS for more than one filter.


def filter_fits_files_by_XRT_filter(fits_files):
    """
    Filters FITS files by XRT filter criteria, accommodating flexible user input formats. It enhances readability and ensures valid input by normalizing filter names.

    Parameters:
    - fits_files (list): List of FITS file paths.

    Returns:
    - filtered_files (list): List of file paths that match the user-selected XRT channel filter.

    Raises:
    - ValueError: If the user input does not match any available filter.
    """
    measurement_info = {}

    for file_path in fits_files:
        try:
            from sunpy.map import Map

            sunpy_map = Map(file_path)
            measurement = sunpy_map.measurement
            # Normalize measurement for consistent comparison
            normalized_measurement = normalize_string(measurement)
            if normalized_measurement in measurement_info:
                measurement_info[normalized_measurement] += 1
            else:
                measurement_info[normalized_measurement] = 1
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    # Display available filters and their counts
    filters_output = "\n".join(
        [f"{key.title()}: {value} files" for key, value in measurement_info.items()]
    )
    print(f"The files have the following filters and counts:\n{filters_output}")
    filter_choice = input("Please select an XRT channel filter of interest: ")
    normalized_filter_choice = normalize_string(filter_choice)

    # Validate user input
    if normalized_filter_choice not in measurement_info:
        raise ValueError(
            "Invalid filter choice. Please enter a valid XRT channel filter from the list provided."
        )

    filtered_files = []
    for file_path in fits_files:
        try:
            sunpy_map = Map(file_path)
            normalized_measurement = normalize_string(sunpy_map.measurement)
            if normalized_filter_choice in normalized_measurement:
                filtered_files.append(file_path)
        except Exception as e:
            print(f"Error processing file {file_path}: {e}")

    name_of_filter = filter_choice.title()
    print(f"\nFilter choice: {name_of_filter}")
    print(
        f"You have {len(filtered_files)} FITS files that match the '{name_of_filter}' filter."
    )
    print("Be sure to store this data as a new variable.")

    return filtered_files

In this example, we'll be investigating the `Al-Poly` filter on the XRT.  

In [ ]:
# Assuming xrt_files_results is a list of FITS file paths
# My new variable name is reflected base on the filter of interest
xrt_Al_poly_Obs_FITs = filter_fits_files_by_XRT_filter(xrt_files_results)


### Filtering FITS Files Through Pixel Size

Filtering Hinode XRT FITS files by pixel dimensions, measured in units of pixels (`pix`), is a practical approach to ensure data uniformity. We refer to these **dimensions** within the context of `sunpy.map.Map`.

XRT images typically have dimensions of `[384x384, 512x512, or 1024x1024]`, although any specific area of the CCD can be read out. For comprehensive details, please refer to the [SolarSoft XRT Analysis Guide](https://xrt.cfa.harvard.edu/resources/documents/XAG/XAG.pdf).


The function `filter_fits_files_by_XRT_dimensions` simplifies the process of selecting FITS files based on pixel dimensions. Upon running this function, you'll encounter a prompt displaying the available dimensions within your dataset. After selecting a preferred option, the function will generate a new list of FITS files filtered according to the specified dimensions. To utilize this functionality, execute the function in the cell below and follow the instructions in the subsequent steps.

In [ ]:
def filter_fits_files_by_XRT_dimensions(fits_files):
    """
    Filters FITS files by their dimensions, allowing for flexible input formatting.
    Provides up to three attempts for input, with error handling for invalid formats.

    Parameters:
    - fits_files (list): List of paths to the FITS files.

    Returns:
    - A list of file paths that match the user-selected dimensions.
    """
    dimension_counts = {}

    # Determine the unique dimensions of all FITS files and count them
    for file_path in fits_files:
        xrt_map = sunpy.map.Map(file_path)
        # Construct a tuple for dimensions to facilitate comparison
        dimensions_tuple = (xrt_map.dimensions.x.value, xrt_map.dimensions.y.value)
        dimensions_str = f"{dimensions_tuple[0]:.0f}, {dimensions_tuple[1]:.0f}"
        dimension_counts[dimensions_str] = dimension_counts.get(dimensions_str, 0) + 1

    print(
        "Unique dimensions (in pixels) found in the dataset and their counts:\n x:pix , y:pix"
    )
    for dimensions, count in sorted(dimension_counts.items()):
        print(f"{dimensions}: {count} files")

    attempts = 0
    while attempts < 3:
        chosen_dimensions_str = input(
            "\nPlease enter the dimensions of interest (e.g., '384, 384'): "
        )
        try:
            x_dim, y_dim = (
                int(dim.strip()) for dim in chosen_dimensions_str.split(",")
            )
            chosen_dimensions_tuple = (x_dim, y_dim)
            chosen_dimensions_str = (
                f"{x_dim}, {y_dim}"  # Reformatted string for comparison
            )
        except ValueError:
            print("Invalid format. Please enter dimensions in the format 'X, Y'.")
            attempts += 1
            continue

        # Check if the processed input matches any known dimensions
        if chosen_dimensions_str in dimension_counts:
            break  # Valid dimension found; proceed with filtering
        else:
            print(f"Error: '{chosen_dimensions_str}' is not a recognized dimension.")
            attempts += 1

    if attempts == 3:
        print(
            "\nMaximum attempts reached. Please rerun and try again. \nIf further issues persist, please email xrtpy@cfa.harvard.edu."
        )
        return []

    filtered_files = []
    for file_path in fits_files:
        xrt_map = sunpy.map.Map(file_path)
        if (
            xrt_map.dimensions.x.value,
            xrt_map.dimensions.y.value,
        ) == chosen_dimensions_tuple:
            filtered_files.append(file_path)

    print(
        f"\nFound {len(filtered_files)} files matching the dimensions {chosen_dimensions_str}."
    )
    return filtered_files

In [ ]:
# For this example, I will continue to use xrt_Al_poly_Obs_FITs. The function will prompt you to select from
# available dimensions and return a new list containing only the files that match the selected dimensions.

# Now, call the function with your list of FITS file paths:
xrt_Al_poly_384_Obs_FITs = filter_fits_files_by_XRT_dimensions(xrt_Al_poly_Obs_FITs)


### Filtering FITS Files by Exposure Time

Different exposure times can significantly affect the quality and the type of data captured in each image. This section introduces a method to filter your dataset based on specific exposure time criteria, allowing for a more refined and targeted analysis.

##### Why Filter by Exposure Time?

Filtering by exposure time can help isolate images that are best suited for your research goals. For instance, shorter exposure times might be preferred for studying rapid solar events, while longer exposures could provide deeper insights into fainter solar features. This flexibility ensures that you can tailor your dataset to include only the most relevant images.

##### How It Works

The `filter_fits_files_by_exposure_time` function enables you to specify criteria for selecting images based on their exposure times. You can choose to include images with an exact exposure time, those with exposure times within a specified range, or images with exposure times above or below a certain threshold. This functionality is designed to give you maximum control over the composition of your dataset.

##### Key Features:
- **Exact Match**: Retrieve images with a specific exposure time by entering the exact value in seconds.
- **Range Selection**: Specify a range of exposure times to include images that fall within this interval.
- **Above or Below a Threshold**: Filter images based on whether their exposure times are above or below a specified value.



In [ ]:
def filter_fits_files_by_exposure_time(fits_files):
    """
    Filters FITS files based on exposure time criteria specified by the user.

    Parameters:
    - fits_files (list): List of FITS file paths.

    Returns:
    - List of FITS file paths that match the exposure time criteria.
    """
    exposure_time_counts = {}
    for file_path in fits_files:
        map_ = sunpy.map.Map(file_path)
        exposure_time = round(map_.exposure_time.to(u.s).value, 2)
        if exposure_time not in exposure_time_counts:
            exposure_time_counts[exposure_time] = {"count": 1, "files": [file_path]}
        else:
            exposure_time_counts[exposure_time]["count"] += 1
            exposure_time_counts[exposure_time]["files"].append(file_path)

    # Display the exposure times and their counts
    print("Exposure times in the dataset (in seconds) and their counts:")
    for time, info in exposure_time_counts.items():
        print(f"{time}s: {info['count']} file(s)")

    # Prompt the user for exposure time criteria
    criteria = input(
        "\nEnter the exposure time criteria (e.g., '0.36' for exactly 0.36s, '<1.5' for less than 1.5s, '>1.5' for more than 1.5s, '2-120' for between 2s and 120s): "
    )
    print("\nSelected exposure time(s):", criteria)
    # Process the user input and filter files
    filtered_files = []
    if "-" in criteria:
        lower, upper = map(float, criteria.split("-"))
        for time, info in exposure_time_counts.items():
            if lower <= time <= upper:
                filtered_files.extend(info["files"])
    elif criteria.startswith("<"):
        max_time = float(criteria[1:])
        for time, info in exposure_time_counts.items():
            if time < max_time:
                filtered_files.extend(info["files"])
    elif criteria.startswith(">"):
        min_time = float(criteria[1:])
        for time, info in exposure_time_counts.items():
            if time > min_time:
                filtered_files.extend(info["files"])
    else:
        exact_time = float(criteria)
        for time, info in exposure_time_counts.items():
            if time == exact_time:
                filtered_files.extend(info["files"])

    print(f"\nFound {len(filtered_files)} files matching the criteria.")
    return filtered_files

To use the `filter_fits_files_by_exposure_time` function, simply run the provided code with your list of FITS files. You will then be prompted to enter your exposure time criteria. The function will process your request and return a filtered list of FITS files that match your specified criteria, making it easier to proceed with a more focused analysis.

In [ ]:
# In this example we're using the filtered Al-Poly Filter & 384 by 384 pixels FITS
xrt_Al_poly_384_Obs_FITs_fixed_exposure_time = filter_fits_files_by_exposure_time(
    xrt_Al_poly_384_Obs_FITs
)


## Navigating FITS Images 

Now, we have the capability to review images within our curated list of filtered FITS files. This enhanced navigation functionality not only facilitates sequential browsing but also introduces the ability to directly access a specific image within the dataset. Such a feature is invaluable for evaluating your dataset in detail, allowing you to identify and exclude FITS files that may not be relevant to your analysis or contain poor-quality images captured by the XRT.

Using the `view_fits_images` function, you can move forward or backward through the image dataset or jump to an image directly navigate to a specific image by entering its number in the dataset. This feature is particularly useful when dealing with large datasets, as it enables quick access to images of interest without the need to sequentially skim through potentially hundreds of files. Whether you're looking for a particular phase of a solar event or need to examine the quality of specific images, this tool enhances your workflow by making data access more efficient and user-friendly.

You may find the need to remove certain images or FITS data that are not relevant or of poor quality for your analysis. To facilitate this, maintain a list with either the index numbers or the titles of the FITS files. This metadata is readily available each time you view an image in the dataset. In a later section titled *[Removing Unwanted FITS Files](#Removing-unwanted-FITS-Files)*, we will introduce functions designed specifically for cleaning your dataset, leveraging the list you've compiled during the review process.

In [ ]:
def view_fits_images(fits_files):
    """
    Displays FITS images one at a time with navigation buttons and an option to jump to a specific image.

    Parameters:
    - fits_files (list): List of paths to the FITS files.
    """

    current_index = [0]  # Use a list to allow modifications from inner functions

    def show_image(index=None):
        """Displays the image at the current index and clears the previous output."""
        if index is not None:
            # Safely update the current index based on user input
            current_index[0] = max(0, min(index, len(fits_files) - 1))

        clear_output(wait=True)  # Clear the previous image and controls
        display(
            widgets.HBox([prev_button, next_button, jump_input, jump_button])
        )  # Redisplay the controls

        fits_path = fits_files[current_index[0]]
        sunpy_map = sunpy.map.Map(fits_path)

        # Display FITS file name and image counter
        print(
            f"File: {fits_path.split('/')[-1]} (Image {current_index[0] + 1} of {len(fits_files)})"
        )

        plt.figure(figsize=(6, 6))
        sunpy_map.plot()
        plt.show()

        update_buttons_status()

    def go_next(btn):
        """Go to the next image."""
        if current_index[0] < len(fits_files) - 1:
            current_index[0] += 1
            show_image()

    def go_prev(btn):
        """Go to the previous image."""
        if current_index[0] > 0:
            current_index[0] -= 1
            show_image()

    def jump_to_image(btn):
        """Jump to the image number entered by the user."""
        try:
            index = int(jump_input.value) - 1  # Convert to 0-based index
            show_image(index)
        except ValueError:
            print("Please enter a valid image number.")

    def update_buttons_status():
        """Updates the status of next/prev buttons based on the current index."""
        next_button.disabled = current_index[0] >= len(fits_files) - 1
        prev_button.disabled = current_index[0] <= 0

    # Create navigation buttons
    next_button = widgets.Button(description="Next")
    prev_button = widgets.Button(description="Previous")
    next_button.on_click(go_next)
    prev_button.on_click(go_prev)

    # Create jump to image widgets
    jump_input = widgets.Text(
        description="Jump to Image:", placeholder="Enter image number"
    )
    jump_button = widgets.Button(description="Go")
    jump_button.on_click(jump_to_image)

    # Initially display buttons and the first image
    display(widgets.HBox([prev_button, next_button, jump_input, jump_button]))
    show_image()

#### Simple and Intuitive Use

To navigate through your FITS images dataset:

1. **Navigation**: Use the "Next" or "Previous" buttons to move sequentially through your dataset, one image at a time.
2. **Direct Jump**: If you want to view a specific image, enter its number in the "Jump to Image" field and click "Go". The image number should correspond to its sequential position in your list of FITS files, allowing direct and straightforward access to any image in your dataset.


In [ ]:
# We're going to use our filtered FITS data set: xrt_Al_poly_384_Obs_FITs
view_fits_images(xrt_Al_poly_384_Obs_FITs_fixed_exposure_time)

### Removing Unwanted FITS Files 

Strategy for Removal
The removal process involves three main steps:

##### Identification: 
While reviewing images, note the index numbers or title filename of those you wish to exclude. Keeping track of these details as you go can streamline the removal process.

##### Compilation: 
Organize the identified indices or filenames into a Python list. This step formalizes your removal list, making the next step straightforward.

##### Execution: 
With the list prepared, you'll use a specific function to remove the unwanted files from your dataset. This function will be based on whether you're working with indices or filenames.

#####  Implementation Details
This section will introduce two functions tailored to your method of identification — one for index numbers and another for filenames. You can choose the one that best fits your workflow.

The emphasis here is on providing a clear and actionable guide for cleaning your dataset, ensuring that the data you work with is precisely what you need for your analysis.

After reviewing the FITS images using the `view_fits_images` function, you may identify some images that you wish to remove from your dataset. This could be due to poor quality, irrelevance to your study, or any other reason. Below we provide a straightforward method to facilitate this, allowing you to easily remove these unwanted files from your dataset.

##### Strategy for Removal

1. **Identification**: While reviewing images, note the index number(s) or FITS filename(s) of those you wish to exclude. 


2. **Compilation**: Create a Python [list](https://docs.python.org/3/tutorial/datastructures.html) with these indices as integers or FITS filename as a string.

3. **Execution**: With the list prepared, you'll use a specific function to remove the unwanted files from your dataset. This function will be based on whether you're working with indices or filenames.

##### Implementation

Depending on your preference for using image numbers or filenames for identification, you can use one of the two provided functions.

##### Using Index Numbers to Remove Files

For cases where you've identified unwanted images by their index position number in the dataset, the `remove_fits_by_index` function will do the removal.

In [ ]:
def remove_fits_by_index(original_list, indices_to_remove):
    """
    Removes FITS files from the dataset based on their index numbers.

    Parameters:
    - original_list (list): The original list of FITS file paths.
    - indices_to_remove (list): A list of index numbers representing the FITS files to remove.

    Returns:
    - A new list with the specified FITS files removed.
    """
    return [
        item for idx, item in enumerate(original_list) if idx not in indices_to_remove
    ]

In [ ]:
# Example usage of the remove_fits_by_index function
indices_to_remove = [
    2,
    5,
    7,
]  # Assuming these are the indices of images to remove, based on their order in the dataset

# Generate the updated list excluding specified FITS files
updated_index_xrt_Al_poly_384_Obs = remove_fits_by_index(
    xrt_Al_poly_384_Obs_FITs_fixed_exposure_time, indices_to_remove
)

# Print the new length of the file list
print("New length of file list:", len(updated_index_xrt_Al_poly_384_Obs))

# Optionally, print the first and last filenames in the updated list to show the range of the dataset
if updated_index_xrt_Al_poly_384_Obs:
    print(
        "First file in the list:", updated_index_xrt_Al_poly_384_Obs[0].split("/")[-1]
    )
    print(
        "Last file in the list:", updated_index_xrt_Al_poly_384_Obs[-1].split("/")[-1]
    )
else:
    print("The list is now empty.")

##### Using FITS filename to remove file
If you prefer noting down filenames:

##### Using FITS Filenames to Remove Files

If you find it more intuitive to work with filenames rather than index numbers, the following method allows you to remove unwanted images by their filenames. This can be particularly useful when the filenames contain descriptive information about the data, making them easier to identify without viewing each image sequentially.


In [ ]:
def remove_fits_by_filename(original_list, filenames_to_remove):
    """
    Removes FITS files from the dataset based on their filenames.

    Parameters:
    - original_list (list): The original list of FITS file paths.
    - filenames_to_remove (list): A list of filenames representing the FITS files to remove.

    Returns:
    - A new list with the specified FITS files removed.
    """
    return [
        item for item in original_list if item.split("/")[-1] not in filenames_to_remove
    ]

In [ ]:
# Example usage of the remove_fits_by_filename function
filenames_to_remove = [
    "l1_xrt20210521_185300_6.fits",
    "l1_xrt20210521_185720_2.fits",
    "l1_xrt20210521_190120_3.fits",
]  # Example filenames of images to remove

# Generate the updated list excluding the specified FITS files
updated_FITS_xrt_Al_poly_384_Obs = remove_fits_by_filename(
    xrt_Al_poly_384_Obs_FITs_fixed_exposure_time, filenames_to_remove
)

# Print the new length of the file list
print("New length of file list:", len(updated_FITS_xrt_Al_poly_384_Obs))

# Optionally, print the first and last filenames in the updated list to show the range of the dataset
if updated_FITS_xrt_Al_poly_384_Obs:
    print("First file in the list:", updated_FITS_xrt_Al_poly_384_Obs[0].split("/")[-1])
    print("Last file in the list:", updated_FITS_xrt_Al_poly_384_Obs[-1].split("/")[-1])
else:
    print("The list is now empty.")


## Visualizing Solar Dynamics: Creating a Movie from XRT FITS Data ☀️

Creating a movie from FITS files is an excellent way to visualize data, particularly for dynamic solar phenomena observed by the Hinode XRT. We can accomplish this using SunPy to handle FITS files and matplotlib, along with imageio, to create the animation.

The `create_solar_movie_from_FITS` function generates a movie (MP4 format) from the FITS files you provide. If you're interested in customizing this process, you can review and modify the function as needed. To proceed with creating a movie using the default settings, run the cell containing the function below. Then, move to the following cell for instructions on how to use this function in your workflow.

In [ ]:
def create_solar_movie_from_FITS(
    fits_files, output_file="solar_movie.mp4", fps=5, processing=True
):
    """
    Creates a movie from a sequence of FITS files.

    Parameters:
    - fits_files (list): List of paths to the FITS files.
    - output_file (str): Path where the output movie will be saved.
    - fps (int): Frames per second for the output movie.
    - processing (bool): If True, prints processing messages. Default is True.
    """
    # Create a temporary directory to store the frames
    frames_dir = tempfile.mkdtemp()
    frames = []

    if processing:
        print(f"Starting to process {len(fits_files)} FITS files.\n")

    for i, file_path in enumerate(fits_files):
        if processing:
            print(f"Processing file {i+1}/{len(fits_files)}: {file_path}")
        # Load the FITS file as a SunPy Map
        xrt_map = sunpy.map.Map(file_path)

        # Plotting the SunPy Map
        fig = plt.figure(figsize=(8, 8))
        ax = plt.subplot(projection=xrt_map)
        xrt_map.plot(axes=ax)

        # Adding title and labels
        plt.title(
            f'XRT {xrt_map.measurement} {xrt_map.date.strftime("%Y-%m-%d %H:%M:%S")}'
        )
        ax.set_xlabel("Helioprojective Longitude (Solar-X)")
        ax.set_ylabel("Helioprojective Latitude (Solar-Y)")
        plt.tight_layout()

        # Saving the frame
        frame_path = os.path.join(frames_dir, f"frame_{i:04d}.png")
        plt.savefig(frame_path)
        plt.close(fig)
        frames.append(frame_path)
        if processing:
            print(f"Saved frame {i+1}/{len(fits_files)}")

    if processing:
        print("\nStarting to compile the movie.\n")

    # Compile the movie from saved frames
    with imageio.get_writer(output_file, fps=fps) as writer:
        for frame_path in frames:
            image = imageio.imread(frame_path)
            writer.append_data(image)

    # Cleanup: Remove temporary frames and directory
    for frame_path in frames:
        os.remove(frame_path)
    os.rmdir(frames_dir)

    if processing:
        print(
            f"\nMovie created: {output_file}\nAll done!\nMake sure to download the movie to your local machine to see it."
        )

In [ ]:
# Specify the output file name for the movie
solar_movie_example = "solar_movie_example.mp4"

# Create the movie from the list of FITS files
# Using our Al-Poly by 384X384 filtered FITs files
create_solar_movie_from_FITS(
    xrt_Al_poly_384_Obs_FITs_fixed_exposure_time,
    output_file=solar_movie_example,
    fps=15,
    processing=True,
)

In [ ]:
# from IPython.display import Video

# Display the created movie within the notebook. Might to add "embed=True" to enable videos directly into a Jupyter Notebook
Video(solar_movie_example)

##### **A Note from Joy, an xrtpy dev**

As an enthusiast of the Hinode X-Ray Telescope (XRT) and a developer for XRTpy, I created this guide to simplify the process of accessing and analyzing Hinode-XRT data using Python. The functions and methods presented in this guide are designed to serve as a starting point, and they are not currently part of the official `xrtpy` library. I encourage you to adapt and modify these functions to fit your specific research needs and preferences.

##### Contributing to XRTpy

If you're interested in contributing to the guide or the XRTpy project itself, we warmly welcome your input and collaboration. You can engage with us through [XRTpy's GitHub repository](https://github.com/HinodeXRT/xrtpy/pulls). Whether it's by suggesting features, improving documentation, or adding new functionalities, your contributions can significantly impact the scientific community's ability to study the Sun.

##### Contact and Sharing

Should you have any questions, or if you wish to share how you've utilized this guide in your work, please don't hesitate to reach out via email at [xrtpy@cfa.harvard.edu](mailto:xrtpy@cfa.harvard.edu). If you share or present your work based on this guide, please ensure to reference it appropriately.

Thank you for exploring the Hinode-XRT data throught this guide and XRTpy. Your feedback and contributions are what make our community thrive.
